In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
from sklearn import svm
from sklearn.metrics import mean_squared_error,roc_curve,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
import tensorflow.keras as keras
from tensorflow.keras.models import model_from_json

In [ ]:
import pickle
from os import path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp drive/MyDrive/diabetes.csv .
diabetes = pd.read_csv("diabetes.csv")

In [ ]:
def loadModels():
  if(path.exists("drive/MyDrive/machineLearning.sav")):
        ml = pickle.load(open("drive/MyDrive/machineLearning.sav", 'rb'))
  else:
        ml = ""
  if(path.exists("drive/MyDrive/Deep")):
        kara = Sequential()
        kara.add(Dense(100, input_dim=8, activation='relu'))
        kara.add(Dropout(0.2))
        kara.add(Dense(2, activation='softmax'))
        kara.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        kara.load_weights("drive/MyDrive/Deep/deeplearning")
  else:
        kara = ""
  return ml,kara
rf,kara = loadModels()

# LECTURE DES DONNES

In [ ]:
diabetes.tail(5)

In [ ]:
diabetes.head(5)


 À PROPOS DU JEUX DE DONNEES

Cet ensemble de données provient à l'origine d’un échantillon publié par un Institut du diabète.

L'objectif de l'ensemble de données est de prédire avec un diagnostique si un patient est atteint du diabète.

Les ensembles de données comprennent plusieurs variables prédictives médicales et une variable cible, « Outcome ». Les variables prédictives comprennent le nombre de grossesses que le patient a eues, son IMC, son taux d'insuline, son âge, etc.

Chaque ligne représente un patient et les colonnes sont :

    Grossesses: nombre de fois enceintes
    Glucose: Concentration en glucose plasmatique 2 heures dans un test de tolérance au glucose par voie orale
    BloodPressure: pression artérielle diastolique (mm Hg)
    SkinThickness: Épaisseur du pli cutané des triceps (mm)
    Insuline: insuline sérique de 2 heures (mu U / ml)
    IMC: indice de masse corporelle (poids en kg / (taille en m) ^ 2)
    DiabetesPedigreeFunction: Fonction pedigree du diabète
    Age: Age (ans)
    Résultat: Variable de classe (0 ou 1).



In [ ]:
diabetes.columns

Afficher le type de donnees des colonnes

In [ ]:
diabetes.dtypes

Combien de diagnostiques on a ?




In [ ]:
diabetes['Outcome'].value_counts()

In [ ]:
diabetes.drop("Outcome",axis=1).hist(figsize=(20,20),bins=15)

**Visualisation des donnees**

In [ ]:
target_0 = diabetes[diabetes.Outcome == 0]
plt.scatter(target_0['Glucose'],target_0['BloodPressure'],color='blue',label='target_0')
target_1 =diabetes[diabetes.Outcome == 1]
plt.scatter(target_1['Glucose'],target_1['BloodPressure'],color='red',label='target_1')
plt.legend()
plt.xlabel('Glucose')
plt.ylabel('BloodPressure')
plt.show()


on peut constater que plus le taux de glucose est elevé, plus l'enregistrement est associé à un diabète (résultat = 1, points rouges), tandis que plus il est bas, et plus il est associé à l'absence du diabète (résultat = 0, points bleus).


---

Il y a un ensemble de points de valeur 0 pour Glucose et un autre de valeur 0 pour BloodPressure. Cela n'a pas de sens physiquement. Il semble que ces données ont été remplies avec 0 alors que la valeur aurait dû être NULL.

In [ ]:
for c in diabetes.columns:
    print('Pour la  colonne ',c,'il y\'a ',diabetes[diabetes[c]==0][c].count(),'zero valeurs.')



Pour certaines de ces colonnes, zéro a un sens, comme pour le nombre de grossesses (Pregnancies) et le résultat (Outcome). Mais pour d'autres colonnes, comme BloodPressure ou BMI, zéro n'a absolument aucun sens

In [ ]:
# example: plot histograms of Age for Outcome=1 and Outcome=0.
plt.figure()
plt.hist(diabetes[diabetes.Outcome==1]['Age'],bins=15,label='Diabetes',color='b',alpha=0.2)
plt.hist(diabetes[diabetes.Outcome==0]['Age'],bins=15,label='No Diabetes',color='g',alpha=0.2)
plt.xlabel('Age')
plt.ylabel('frequency')
plt.legend()
plt.show()

**Divisons de l'ensemble de donnees en dataTest et dataTrain**

In [ ]:
np.random.seed(666)
X = diabetes.drop("Outcome",axis=1)
Y = diabetes["Outcome"]
scaler = StandardScaler()
X = scaler.fit_transform(X)
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2)

In [ ]:
print("Dimension de notre dataTrain",x_train.shape)
print("Dimension de notre dataTest",x_train.shape)

# LOGISTIC REGRESSION

In [ ]:
mod = LogisticRegression(max_iter=5000)
parameters = { 'solver': ['newton-cg', 'lbfgs', 'liblinear','saga'],
               'penalty':['none', 'l1', 'l2', 'elasticnet'],
               'C':[1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
               } 
search = GridSearchCV(mod, parameters, scoring='accuracy', n_jobs=-1, cv=5)
result = search.fit(x_train,y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)
params1 = result.best_params_

Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'}

In [ ]:
lr=LogisticRegression(C=params1['C'],solver=params1['solver'],penalty=params1['penalty'],max_iter=5000)
lr.fit(x_train,y_train)

In [ ]:
y_pred_logistic=lr.predict(x_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_logistic)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

In [ ]:
#taux de succès
acc = metrics.accuracy_score(y_test,y_pred_logistic)
err=1.0-acc
#sensibilité (ou rappel)
se = metrics.recall_score(y_test,y_pred_logistic)
print("Le rappel est : {:.3f}".format(se))
print("L erreur est : {:.3f}".format(err))
print("L accuracy est : {:.3f}".format(acc))
print('Training set Accuracy :{:.3f}'.format(lr.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(lr.score(x_test,y_test)))

In [ ]:
print('predictions:{}'.format(list(y_pred_logistic[0:20])))
print('true       :{}'.format(list(y_test[0:20].astype(int))))

In [ ]:
scores_logistic = cross_val_score(lr, x_train, y_train, cv=5,scoring='accuracy')
print("Cross-validation scores: {}".format(scores_logistic))
print("Moyenne des Taux de Success:{}".format(scores_logistic.mean()))

In [ ]:
#calcul des probas d'affectation sur ech test
probas_logistic =lr.predict_proba(x_test)
probas_logistic[0:20,:]

In [ ]:
#score de 'presence'
score_logistic=probas_logistic[:,1]
#transf en 0/1 de Y_test
pos=pd.get_dummies(y_test).values
pos[0:20,:]

# **SVR**

De nombreux algorithmes de
machine learning reposent sur des paramètres
qui ne sont pas toujours évidents à déterminer
pour obtenir les meilleurs performances sur un
jeu de données à traiter. Ex. SVM.

In [ ]:
parametres = [{'C':[0.1,1,10],'kernel':['rbf','linear']}]
grid = GridSearchCV(estimator=svm.SVC(),param_grid=parametres,scoring='accuracy',cv=5)
grid.fit(x_train,y_train)

Grille de recherche. On
indique les paramètres à faire varier,
scikit-learn les croise et mesure les
performances en validation croisée

In [ ]:
C = grid.best_params_['C']
kernel = grid.best_params_['kernel']

In [ ]:
classifier = svm.SVC(C=C,kernel=kernel)
classifier.fit(x_train,y_train)

In [ ]:
y_pred_svr = classifier.predict(x_test)
#taux de succès
acc_svr = metrics.accuracy_score(y_test,y_pred_svr)
err_svr=1.0-acc_svr
#sensibilité (ou rappel)
se_svr = metrics.recall_score(y_test,y_pred_svr)
print("Le rappel est : {:.3f}".format(se_svr))
print("L erreur est : {:.3f}".format(err_svr))
print("L accuracy est : {:.3f}".format(acc_svr))
print('Training set Accuracy :{:.3f}'.format(classifier.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(classifier.score(x_test,y_test)))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_svr)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

# KNN

In [ ]:
parametres = {'n_neighbors':np.arange(2,25,1)}
grid = GridSearchCV(estimator=KNeighborsClassifier(),param_grid=parametres,scoring='accuracy',cv=5)
grid.fit(x_train,y_train)

In [ ]:
n_neighbors = grid.best_params_['n_neighbors']

cette courbe nous montre la variance de l'erreur par rapport au nombre de neighbors pris.
Comme on le constate le nombre optimal a prendre est 11

In [ ]:
modelKNN= KNeighborsClassifier(n_neighbors=n_neighbors, weights='uniform')

maintenant on va entrainer notre model

In [ ]:
modelKNN.fit(x_train,y_train)

In [ ]:
y_pred_knn = modelKNN.predict(x_test)
#taux de succès
acc_knn = metrics.accuracy_score(y_test,y_pred_knn)
err_knn=1.0-acc_knn
#sensibilité (ou rappel)
se_knn = metrics.recall_score(y_test,y_pred_knn)
print("Le rappel est : {:.3f}".format(se_knn))
print("L erreur est : {:.3f}".format(err_knn))
print("L accuracy est : {:.3f}".format(acc_knn))
print('Training set Accuracy :{:.3f}'.format(modelKNN.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(modelKNN.score(x_test,y_test)))

In [ ]:
scores_knn = cross_val_score(modelKNN, x_train, y_train, cv=5)
print("Cross-validation scores: {}".format(scores_knn))
print("Moyenne des Taux de Success:{}".format(scores_knn.mean()))

In [ ]:
y_pred_knn=modelKNN.predict(x_test)
cm = metrics.confusion_matrix(y_test,y_pred_knn)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

# **RandomForest**

In [ ]:
parametres = {'max_depth':np.arange(3,10,1),'n_estimators':np.arange(4,200,1)}
grid = GridSearchCV(estimator=RandomForestClassifier(),param_grid=parametres,scoring='accuracy',cv=5)
grid.fit(x_train,y_train)

In [ ]:
rf = RandomForestClassifier(max_depth=5,n_estimators=182, random_state=0)
rf.fit(x_train, y_train)

Adjustons les max_features param , pour voir si le resultat s'ameliore


In [ ]:
y_pred_random = rf.predict(x_test)
#taux de succès
acc_random = metrics.accuracy_score(y_test,y_pred_random)
err_random=1.0-acc_random
#sensibilité (ou rappel)
se_random = metrics.recall_score(y_test,y_pred_random)
print("Le rappel est : {:.3f}".format(se_random))
print("L erreur est : {:.3f}".format(err_random))
print("L accuracy est : {:.3f}".format(acc_random))
print('Training set Accuracy :{:.3f}'.format(rf.score(x_train,y_train)))
print('Test set Accuracy :{:.3f}'.format(rf.score(x_test,y_test)))

In [ ]:
y_pred_random=rf.predict(x_test)
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_random)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

# DEEP LEARNING 

In [ ]:
def getTheBestNeural():
    scores = []
    for i in np.arange(5,100,1):
      print("i===> {}".format(i))
      stopping_val_loss =EarlyStopping(monitor="val_loss",patience=70,mode="min",baseline=0.42)
      stopping_val_accuracy =EarlyStopping(monitor="val_accuracy",patience=70,mode="max")
      seed = 7
      np.random.seed(seed)
      kara = Sequential()
      kara.add(Dense(i,input_dim=8, activation='relu'))
      kara.add(Dropout(0.2))
      kara.add(Dense(2, activation='softmax'))
      kara.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
      history = kara.fit(x_train, y_train1, epochs=200, batch_size=15,validation_data=(x_test,y_test1),callbacks=[stopping_val_loss,stopping_val_accuracy],verbose=0) # 150 epoch, 10 batch size, verbose = 2
      score = kara.evaluate(x_test,y_test1)
      score.append(i)
      scores.append(score)
    errors = []
    for element in scores:
      errors.append(element[0])
    meanError = np.mean(errors)
    max = scores[0]
    for j in range(len(scores)):
      if scores[j][0] <= meanError:
        if scores[j][1] > max[1]:
          max = scores[j] 
    return max
bestones = getTheBestNeural()

In [ ]:
y_test1 = pd.get_dummies(y_test)
y_train1 = pd.get_dummies(y_train)

In [ ]:
stopping_val_loss =EarlyStopping(monitor="val_loss",patience=100,mode="min")
stopping_val_accuracy =EarlyStopping(monitor="val_accuracy",patience=100,mode="max")
seed = 7
np.random.seed(seed)
kara = Sequential()
kara.add(Dense(100,input_dim=8, activation='relu'))
kara.add(Dropout(0.2))
kara.add(Dense(2, activation='softmax'))
kara.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = kara.fit(x_train, y_train1, epochs=150, batch_size=40,validation_data=(x_test,y_test1),callbacks=[stopping_val_loss,stopping_val_accuracy],verbose=2) # 150 epoch, 10 batch size, verbose = 2
score = kara.evaluate(x_test,y_test1)
print(score)

In [ ]:

#display accuracy and loss
epochs = range(len(history.epoch))

weights = kara.layers[0].get_weights()[0]
biases = kara.layers[0].get_weights()[1]

w1 = weights[0][0] #a
w2 = weights[1][0] #b
w3 = weights[2][0] #c
b = biases[0]      #d

#construct hyperplane: ax + by + cz = d
a,b,c,d = w1,w2,w3,b

x_min = diabetes.BMI.min()
x_max = diabetes.BMI.max()

x = np.linspace(x_min, x_max, 100)

y_min = diabetes.DiabetesPedigreeFunction.min()
y_max = diabetes.DiabetesPedigreeFunction.max()

y = np.linspace(y_min, y_max, 100)

Xs,Ys = np.meshgrid(x,y)
Zs = (d - a*Xs - b*Ys) / c

fig = plt.figure(num=None, figsize=(9, 9), dpi=100, facecolor='w', edgecolor='k')
ax = fig.gca(projection='3d')

ax.plot_surface(Xs, Ys, Zs, alpha=0.45)

ax.scatter(diabetes.BMI, diabetes.DiabetesPedigreeFunction, diabetes.Glucose, c=diabetes.Outcome)

ax.set_xlabel('BMI')
ax.set_ylabel('DiabetesPedigreeFunction')
ax.set_zlabel('Glucose')

In [ ]:
# Loss Curves
plt.figure(figsize=[14,8])
plt.plot(history.history['loss'],'b',linewidth=3.0)
plt.plot(history.history['val_loss'],'r',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)

In [ ]:
plt.figure(figsize=[14,8])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
# Fit with cross validation
#estimator = KerasClassifier(build_fn=kara, epochs=150, batch_size=10,verbose=0)
#kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
#results = cross_val_score(kara, X, Y, cv=5)

In [ ]:
predictions_deep=kara.predict(x_test)
y_pred_deep = []
for i in predictions_deep:
  y_pred_deep.append(np.argmax(i))

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=y_pred_deep)
ax= plt.subplot()
sns.heatmap(cm, annot=True, ax = ax, cbar=False)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['target=0', 'target=1'])
ax.yaxis.set_ticklabels(['target=0', 'target=1'])
plt.show()

In [ ]:
score = kara.evaluate(x_test,y_test1)

In [ ]:
plt.scatter(x_test[:,1],y_test,color='blue',label='x_test true')
plt.scatter(x_test[:,1],y_pred_deep,color='red',label='x_test predit')
plt.legend()
plt.xlabel('Glucose')
plt.ylabel('Diabete')
plt.show()

In [ ]:
#taux de succès
acc_deep = metrics.accuracy_score(y_test,y_pred_deep)
err_deep=1.0-acc_deep
#sensibilité (ou rappel)
se_deep = metrics.recall_score(y_test,y_pred_deep)
print("Le rappel est : {:.3f}".format(se_deep))
print("L erreur est : {:.3f}".format(err_deep))
print("L accuracy est : {:.3f}".format(acc_deep))

scoretrain = kara.evaluate(x_train,y_train1)
scoretest = kara.evaluate(x_test,y_test1)
print('Training set Accuracy :{:.3f}'.format(scoretrain[1]))
print('Test set Accuracy :{:.3f}'.format(scoretest[1]))

In [ ]:
print("Score deep learning : {} and error  {}".format(score[0],score[1]))

In [ ]:
fpr, tpr, thr = metrics.roc_curve(y_test, lr.decision_function(x_test))
plt.plot(fpr,tpr, label='logistic regression')
fpr2, tpr2, thr2 = metrics.roc_curve(y_test, classifier.decision_function(x_test))
plt.plot(fpr2,tpr2,label='svm classifier')
fpr1, tpr1, thr1 = metrics.roc_curve(y_test, modelKNN.predict_proba(x_test)[:,1])
plt.plot(fpr1,tpr1,label='knn classifier')
fpr3, tpr3, thr3 = metrics.roc_curve(y_test, rf.predict_proba(x_test)[:,1])
plt.plot(fpr3,tpr3,label='Rndome Forest classifier')
fpr4, tpr4, thr4 = metrics.roc_curve(y_test, kara.predict(x_test)[:,1])
plt.plot(fpr4,tpr4,label='Reseau neuron')
plt.title('roc curve')
plt.legend(loc = 'lower right', fontsize = 16)

# Exporting data

In [ ]:
#Export machine learning
pickle.dump(rf,open("drive/MyDrive/machineLearning.sav","wb"))

In [ ]:
model_json = kara.to_json()
with open('drive/MyDrive/deeplearning.json', "w") as json_file:
    json_file.write(model_json)

In [ ]:
#Export machine learning
pickle.dump(scaler,open("drive/MyDrive/myscaler","wb"))